## BERT（2018年）

BERT（Bidirectional Encoder Representations from Transformers）是一种基于Transformer架构的预训练语言模型，由Google在2018年提出。它在自然语言处理（NLP）领域取得了革命性的进展，广泛应用于各种任务，如文本分类、命名实体识别、问答系统等。BERT 的核心创新是其训练方法，它允许模型学习到双向的上下文信息，这与传统的单向语言模型（如 Word2Vec 或 GloVe）相比是一个显著的进步。BERT 的提出标志着 NLP 领域的一个重要里程碑，它推动了预训练语言模型的发展，并为各种 NLP 任务提供了强大的基础。

### **1. 核心特点**

- **双向编码器**：BERT通过Transformer的编码器结构，能够同时考虑上下文信息，而不仅仅是单向的（如传统的RNN或LSTM）。这使得BERT能够更好地理解语言的语义。
- **预训练+微调**：BERT采用两阶段训练方式：
  1. **预训练阶段**：在大规模无标签文本数据上进行训练，学习通用的语言表示。
  2. **微调阶段**：在特定任务上使用少量标注数据进行微调，以适应具体任务。
- **多任务学习**：BERT在预训练阶段使用了两个任务：
  1. **Masked Language Model (MLM)**：随机遮蔽输入文本中的某些词，模型需要预测被遮蔽的词。
  2. **Next Sentence Prediction (NSP)**：判断两个句子是否是连续的上下文关系。

### **2. 模型架构**

- **Transformer编码器**：BERT基于Transformer的编码器部分，具有多头自注意力机制和前馈神经网络。
- **多层结构**：BERT有多个编码器层（如BERT-base有12层，BERT-large有24层），每层都包含自注意力机制和前馈网络。
- **输入表示**：
  - 输入是token序列，通常使用WordPiece分词。
  - 输入包括三个部分：
    1. **Token Embeddings**：词嵌入。
    2. **Segment Embeddings**：用于区分句子对中的两个句子。
    3. **Position Embeddings**：用于表示词的位置信息。

### **3. 预训练任务**

- **Masked Language Model (MLM)**：
  - 随机遮蔽输入序列中的15%的词。
  - 模型需要预测被遮蔽的词。
  - 这种任务使得模型能够学习到上下文信息。
- **Next Sentence Prediction (NSP)**：
  - 输入是两个句子，模型需要判断第二个句子是否是第一个句子的下一句。
  - 这种任务帮助模型理解句子之间的关系。

### **4. 微调阶段**

- 在特定任务上，BERT的输出层会被替换为任务相关的输出层（如分类层、序列标注层等）。
- 使用少量标注数据对整个模型进行微调，以适应具体任务。

### **5. 优势**

- **强大的语言理解能力**：BERT能够捕捉复杂的语言模式，因为它同时考虑了上下文信息。
- **广泛适用性**：BERT可以应用于多种NLP任务，如文本分类、情感分析、问答系统等。
- **高性能**：在多个NLP基准测试中，BERT表现优异，显著超越了之前的模型。

### **6. 变体**

- **BERT-base**：12层Transformer编码器，768维隐藏层，12个注意力头。
- **BERT-large**：24层Transformer编码器，1024维隐藏层，16个注意力头。
- **其他变体**：
  - **RoBERTa**：改进了BERT的预训练策略，去掉了NSP任务，增加了训练数据和训练步数。
  - **ALBERT**：通过参数共享和更小的嵌入层，减少了模型的参数量。
  - **DistilBERT**：通过知识蒸馏，构建了一个更小的BERT模型，但保留了大部分性能。

### **7. 应用场景**

- **文本分类**：如情感分析、垃圾邮件检测。
- **命名实体识别**：如提取文本中的地名、人名等。
- **问答系统**：如SQuAD数据集上的问答任务。
- **机器翻译**：虽然BERT本身不是为翻译设计，但可以作为特征提取器。
- **文本生成**：结合生成模型（如GPT）进行文本生成任务。

BERT通过双向Transformer编码器和创新的预训练任务，显著提升了自然语言处理任务的性能。它的成功推动了后续一系列基于Transformer的模型（如GPT、T5、BART等）的发展，成为现代NLP领域的基石。

项目：商品评论情感分析

In [19]:
import time
import datetime
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup

# 检查是否有可用的GPU，如果有则使用GPU，否则使用CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
SEED = 123

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [23]:
# 读取文件内容的函数
def data_read(filepath):
    # 打开指定路径的文件，使用gb18030编码，读取每一行并去除行尾的空白字符
    data_raw = [line.strip() for line in open(filepath, encoding='gb18030')]
    return data_raw  # 返回读取到的数据列表

# 合并数据的函数，save_flag参数用于控制是否保存合并后的数据
def merge_data(save_flag=None):
    # 定义需要处理的文件名列表
    files_name = ['hotel']
    # 初始化正向文本和负向文本的列表
    pos_text, neg_text = [], []
    # 遍历文件名列表
    for filename in files_name:
        # 读取正向文本文件
        pos_temp = data_read('./datasets/txt_cls/' + filename + '/pos.txt')
        # 读取负向文本文件
        neg_temp = data_read('./datasets/txt_cls/' + filename + '/neg.txt')
        # 将读取到的正向文本添加到pos_text列表中
        pos_text.extend(pos_temp)
        # 将读取到的负向文本添加到neg_text列表中
        neg_text.extend(neg_temp)
    # 如果save_flag为True，则将合并后的正向和负向文本保存到指定文件中
    if save_flag:
        write_txt('./datasets/txt_cls/pos_all.txt', pos_text)
        write_txt('./datasets/txt_cls/neg_all.txt', neg_text)
    # 返回合并后的正向和负向文本列表
    return pos_text, neg_text

# 调用merge_data函数，获取正向和负向文本数据
pos_text, neg_text = merge_data()
# 将正向和负向文本数据合并为一个总的句子列表
sentences = pos_text + neg_text

# 设定标签，正向文本标签为0，负向文本标签为1
pos_targets = np.zeros((len(pos_text)))  # 正向文本标签数组，长度与pos_text相同，值全为0
neg_targets = np.ones((len(neg_text)))   # 负向文本标签数组，长度与neg_text相同，值全为1
# 将正向和负向标签数组拼接在一起，并调整形状为(n, 1)
targets = np.concatenate((pos_targets, neg_targets), axis=0).reshape(-1, 1)  # (10000, 1)
# 将标签数组转换为PyTorch张量
total_targets = torch.tensor(targets)

In [24]:
# 从预训练的BERT模型权重中加载中文BERT的分词器
tokenizer = BertTokenizer.from_pretrained('./weights/bert-base-chinese')

# 打印正向文本数据中的第3条文本（索引为2）
print(pos_text[2])

# 使用分词器对第3条文本进行分词，并打印分词结果
print(tokenizer.tokenize(pos_text[2]))

# 使用分词器对第3条文本进行编码（转换为BERT模型的输入ID），并打印编码结果
print(tokenizer.encode(pos_text[2]))

# 使用分词器将编码后的ID转换回分词形式，并打印结果
print(tokenizer.convert_ids_to_tokens(tokenizer.encode(pos_text[2])))

不错，下次还考虑入住。交通也方便，在餐厅吃的也不错。
['不', '错', '，', '下', '次', '还', '考', '虑', '入', '住', '。', '交', '通', '也', '方', '便', '，', '在', '餐', '厅', '吃', '的', '也', '不', '错', '。']
[101, 679, 7231, 8024, 678, 3613, 6820, 5440, 5991, 1057, 857, 511, 769, 6858, 738, 3175, 912, 8024, 1762, 7623, 1324, 1391, 4638, 738, 679, 7231, 511, 102]
['[CLS]', '不', '错', '，', '下', '次', '还', '考', '虑', '入', '住', '。', '交', '通', '也', '方', '便', '，', '在', '餐', '厅', '吃', '的', '也', '不', '错', '。', '[SEP]']


In [25]:
# 将每一句文本转换为数字编码的函数
def convert_text_to_token(tokenizer, sentence, limit_size=50):
    # 使用分词器对句子进行编码，并截断到limit_size长度
    tokens = tokenizer.encode(sentence[:limit_size])
    # 如果编码后的长度小于limit_size + 2（加上首尾标识符），则用0补齐
    if len(tokens) < limit_size + 2:
        tokens.extend([0] * (limit_size + 2 - len(tokens)))
    return tokens  # 返回处理后的编码列表

# 对所有句子进行编码转换，生成输入ID列表
input_ids = [convert_text_to_token(tokenizer, sen) for sen in sentences]
# 将输入ID列表转换为PyTorch张量
input_tokens = torch.tensor(input_ids)

# 打印输入张量的形状
print(input_tokens.shape)  # 输出形状为 [10000, 52]，表示10000个句子，每个句子长度为52
# 打印输入张量的内容
print(input_tokens)

torch.Size([10000, 52])
tensor([[ 101, 6983, 2421,  ..., 2791, 3198,  102],
        [ 101, 1765, 4415,  ..., 7313, 7599,  102],
        [ 101,  679, 7231,  ...,    0,    0,    0],
        ...,
        [ 101, 2769, 2697,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ..., 1218, 1447,  102],
        [ 101, 5439,  782,  ...,  102,    0,    0]])


In [26]:
# 创建注意力掩码的函数
def attention_masks(input_ids):
    atten_masks = []  # 初始化注意力掩码列表
    # 遍历输入ID列表
    for seq in input_ids:
        # 生成当前句子的注意力掩码
        # 如果当前位置的ID大于0（表示不是PAD符号），则为1，否则为0
        seq_mask = [float(i > 0) for i in seq]
        atten_masks.append(seq_mask)  # 将当前句子的掩码添加到列表中
    return atten_masks  # 返回注意力掩码列表

# 调用函数生成注意力掩码
atten_masks = attention_masks(input_ids)
# 将注意力掩码列表转换为PyTorch张量
attention_tokens = torch.tensor(atten_masks)

In [27]:
# 划分训练集和测试集
# 将输入数据和标签数据按照8:2的比例划分为训练集和测试集
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_tokens, total_targets, random_state=666, test_size=0.2)

# 同时划分注意力掩码数据，保持与输入数据的一致性
train_masks, test_masks, _, _ = train_test_split(
    attention_tokens, input_tokens, random_state=666, test_size=0.2)

# 打印训练集和测试集输入数据的形状
print(train_inputs.shape, test_inputs.shape)  # 输出训练集和测试集输入数据的形状

# 打印训练集注意力掩码的形状
print(train_masks.shape)  # 输出训练集注意力掩码的形状

# 打印训练集第一个样本的输入数据
print(train_inputs[0])

# 打印训练集第一个样本的注意力掩码
print(train_masks[0])

torch.Size([8000, 52]) torch.Size([2000, 52])
torch.Size([8000, 52])
tensor([  101,  2769,  6370,  4638,  3221, 10189,  1039,  4638,   117,   852,
         2769,  6230,  2533,  8821,  1039,  4638,  7599,  3419,  3291,  1962,
          671,   763,   117,  3300,   671,  2476,  1377,   809,  1288,  1309,
         4638,  3763,  1355,   119,  2456,  6379,  1920,  2157,  6370,  3249,
         6858,  7313,   106,   102,     0,     0,     0,     0,     0,     0,
            0,     0])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])


In [28]:
# 定义超参数
BATCH_SIZE = 4  # 批处理大小
LEARNING_RATE = 2e-5  # 学习率
WEIGHT_DECAY = 1e-2  # 权重衰减（L2正则化）
EPSILON = 1e-8  # 优化器中的epsilon参数，用于数值稳定性

# 创建训练集的TensorDataset
# 将输入数据、注意力掩码和标签打包成一个数据集
train_data = TensorDataset(train_inputs, train_masks, train_labels)

# 创建训练集的随机采样器
# 随机打乱训练数据，以提高训练效果
train_sampler = RandomSampler(train_data)

# 创建训练集的DataLoader
# 使用随机采样器和批处理大小，将数据分批加载
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

# 创建测试集的TensorDataset
# 将输入数据、注意力掩码和标签打包成一个数据集
test_data = TensorDataset(test_inputs, test_masks, test_labels)

# 创建测试集的顺序采样器
# 按顺序加载测试数据，保持数据顺序不变
test_sampler = SequentialSampler(test_data)

# 创建测试集的DataLoader
# 使用顺序采样器和批处理大小，将数据分批加载
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [29]:
# 遍历训练集的DataLoader，获取一个批次的数据
for i, (train, mask, label) in enumerate(train_dataloader):
    # 打印当前批次中输入数据、注意力掩码和标签的形状
    print(train.shape, mask.shape, label.shape)  # 输出形状为 [BATCH_SIZE, 52], [BATCH_SIZE, 52], [BATCH_SIZE, 1]
    break  # 只取第一个批次的数据，然后退出循环

# 打印训练集DataLoader的长度，即批次数
print('len(train_dataloader)=', len(train_dataloader))

torch.Size([4, 52]) torch.Size([4, 52]) torch.Size([4, 1])
len(train_dataloader)= 2000


In [30]:
# 从预训练的BERT模型权重中加载用于序列分类的BERT模型
# 设置num_labels为2，表示这是一个二分类任务（好评和差评）
model = BertForSequenceClassification.from_pretrained("./weights/bert-base-chinese", num_labels=2)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./weights/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [31]:
# 定义优化器
# 使用AdamW优化器，适用于BERT模型的优化
# 参数lr为学习率，eps为数值稳定性参数（默认值为1e-8）
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=EPSILON)

# 定义训练的轮数
epochs = 2

# 计算总的训练步数
# 总步数 = 批次数 * 训练轮数
total_steps = len(train_dataloader) * epochs

# 设计学习率调度器
# 使用线性学习率调度器，包含预热阶段
# num_warmup_steps为预热步数，num_training_steps为总训练步数
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [32]:
# 定义二分类准确率的计算函数
def binary_acc(preds, labels):  # preds.shape=(4, 2) labels.shape=torch.Size([4, 1])
    # 计算预测结果的最大值索引（即预测的类别）
    # torch.max(preds, dim=1)[1] 返回每行最大值的索引，shape为 [4]
    # labels.flatten() 将标签展平为一维张量，shape为 [4]
    correct = torch.eq(torch.max(preds, dim=1)[1], labels.flatten()).float()
    # 计算准确率：正确预测的数量 / 总样本数
    acc = correct.sum().item() / len(correct)
    return acc  # 返回准确率


def format_time(elapsed):
    # 将时间四舍五入为整数
    elapsed_rounded = int(round((elapsed)))
    # 将时间格式化为 hh:mm:ss 形式
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [33]:
# 定义训练函数
def train(model, optimizer):
    t0 = time.time()  # 记录训练开始时间
    avg_loss, avg_acc = [], []  # 初始化平均损失和平均准确率列表

    model.train()  # 将模型设置为训练模式

    # 遍历训练数据加载器中的每个批次
    for step, batch in enumerate(train_dataloader):
        # 每隔100个批次，输出当前的训练进度和已用时间
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f'  Batch {step:>5,}  of  {len(train_dataloader):>5,}.    time: {elapsed:}.')

        # 将批次数据移动到指定设备（GPU或CPU）
        b_input_ids, b_input_mask, b_labels = (
            batch[0].long().to(device),  # 输入ID
            batch[1].long().to(device),  # 注意力掩码
            batch[2].long().to(device)   # 标签
        )

        # 前向传播：获取模型的输出
        output = model(
            b_input_ids,
            token_type_ids=None,  # 对于BERT模型，token_type_ids通常为None
            attention_mask=b_input_mask,  # 注意力掩码
            labels=b_labels  # 真实标签
        )
        loss, logits = output[0], output[1]  # 获取损失和预测结果

        avg_loss.append(loss.item())  # 记录当前批次的损失

        # 计算当前批次的准确率
        acc = binary_acc(logits, b_labels)
        avg_acc.append(acc)  # 记录当前批次的准确率

        optimizer.zero_grad()  # 清空梯度
        loss.backward()  # 反向传播，计算梯度
        clip_grad_norm_(model.parameters(), 1.0)  # 梯度裁剪，防止梯度爆炸
        optimizer.step()  # 更新模型参数
        scheduler.step()  # 更新学习率

    # 计算平均准确率和平均损失
    avg_acc = np.array(avg_acc).mean()
    avg_loss = np.array(avg_loss).mean()

    return avg_loss, avg_acc  # 返回平均损失和平均准确率

In [34]:
# 定义评估函数
def evaluate(model):
    avg_acc = []  # 初始化平均准确率列表
    model.eval()  # 将模型设置为评估模式

    # 禁用梯度计算，以节省内存和提高速度
    with torch.no_grad():
        # 遍历测试数据加载器中的每个批次
        for batch in test_dataloader:
            # 将批次数据移动到指定设备（GPU或CPU）
            b_input_ids, b_input_mask, b_labels = (
                batch[0].long().to(device),  # 输入ID
                batch[1].long().to(device),  # 注意力掩码
                batch[2].long().to(device)   # 标签
            )

            # 前向传播：获取模型的输出
            output = model(
                b_input_ids,
                token_type_ids=None,  # 对于BERT模型，token_type_ids通常为None
                attention_mask=b_input_mask  # 注意力掩码
            )

            # 计算当前批次的准确率
            acc = binary_acc(output[0], b_labels)
            avg_acc.append(acc)  # 记录当前批次的准确率

    # 计算平均准确率
    avg_acc = np.array(avg_acc).mean()
    return avg_acc  # 返回平均准确率

In [35]:
# 训练和评估的主循环
for epoch in range(epochs):
    # 训练模型
    train_loss, train_acc = train(model, optimizer)
    # 打印当前训练轮次的训练准确率和损失
    print('epoch={}, train_acc={}, loss={}'.format(epoch, train_acc, train_loss))
    
    # 评估模型在测试集上的表现
    test_acc = evaluate(model)
    # 打印当前训练轮次的测试准确率
    print("epoch={}, test_acc={}".format(epoch, test_acc))

  Batch   100  of  2,000.    time: 0:00:05.
  Batch   200  of  2,000.    time: 0:00:09.
  Batch   300  of  2,000.    time: 0:00:14.
  Batch   400  of  2,000.    time: 0:00:18.
  Batch   500  of  2,000.    time: 0:00:22.
  Batch   600  of  2,000.    time: 0:00:27.
  Batch   700  of  2,000.    time: 0:00:31.
  Batch   800  of  2,000.    time: 0:00:35.
  Batch   900  of  2,000.    time: 0:00:40.
  Batch 1,000  of  2,000.    time: 0:00:44.
  Batch 1,100  of  2,000.    time: 0:00:48.
  Batch 1,200  of  2,000.    time: 0:00:53.
  Batch 1,300  of  2,000.    time: 0:00:57.
  Batch 1,400  of  2,000.    time: 0:01:01.
  Batch 1,500  of  2,000.    time: 0:01:06.
  Batch 1,600  of  2,000.    time: 0:01:10.
  Batch 1,700  of  2,000.    time: 0:01:15.
  Batch 1,800  of  2,000.    time: 0:01:19.
  Batch 1,900  of  2,000.    time: 0:01:24.
epoch=0, train_acc=0.87825, loss=0.4390542203724908
epoch=0, test_acc=0.9135
  Batch   100  of  2,000.    time: 0:00:04.
  Batch   200  of  2,000.    time: 0:00:09.

In [36]:
# 定义预测函数
def predict(sen):
    # 将输入文本转换为BERT模型的输入ID
    input_id = convert_text_to_token(tokenizer, sen)
    # 将输入ID转换为PyTorch张量，并移动到指定设备（GPU或CPU）
    input_token = torch.tensor(input_id).long().to(device)  # torch.Size([128])
    
    # 创建注意力掩码
    atten_mask = [float(i > 0) for i in input_id]
    # 将注意力掩码转换为PyTorch张量，并移动到指定设备（GPU或CPU）
    attention_token = torch.tensor(atten_mask).long().to(device)  # torch.Size([128])
    
    # 前向传播：获取模型的输出
    # 将输入张量和注意力掩码调整为形状 [1, 128]，以匹配模型的输入要求
    output = model(
        input_token.view(1, -1),  # 输入ID
        token_type_ids=None,      # 对于BERT模型，token_type_ids通常为None
        attention_mask=attention_token.view(1, -1)  # 注意力掩码
    )
    
    # 打印模型的输出（预测结果）
    print(output[0])
    
    # 返回预测结果的最大值索引（即预测的类别）
    return torch.max(output[0], dim=1)[1]

In [39]:
label = predict('你家的外卖狗都不吃！')
print('好评' if label==0 else '差评')

tensor([[-4.4393,  4.0700]], device='cuda:0', grad_fn=<AddmmBackward0>)
差评
